In [48]:
import fmpsdk as fmp
import pandas as pd
import numpy as np
import os
from dotenv import load_dotenv
import datetime
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum

In [2]:
import numpy as np
import tensorflow as tf

In [3]:
from transformers import pipeline, AutoTokenizer, TFAutoModelForSequenceClassification

c:\Users\zm_if\anaconda3\envs\bigdata1\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
model = TFAutoModelForSequenceClassification.from_pretrained("mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis")
tokenizer = AutoTokenizer.from_pretrained("mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis")



Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaForSequenceClassification: ['roberta.embeddings.position_ids']
- This IS expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFRobertaForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


In [6]:
spark = SparkSession.builder.appName("crypt_cot") \
    .config("spark.driver.bindAddress", "0.0.0.0") \
    .getOrCreate()

In [14]:
workbench_url = "jdbc:mysql://localhost:3306/finance_out?permitMysqlSchema"
driver = "com.mysql.jdbc.Driver"
data_base_crypto = "new_crypto_data"
data_base_cot = "cot_report"
table = "btc_data"
cot_btc_table = "BT"
mssql_url = f"jdbc:sqlserver://ZAHRA\SQLEXPRESS:61254;database={data_base_crypto};trustServerCertificate=true;encrypt=true"
mssql_cot_url = f"jdbc:sqlserver://ZAHRA\SQLEXPRESS:61254;database={data_base_cot};trustServerCertificate=true;encrypt=true"

In [16]:
cot_data = spark.read.format("jdbc") \
    .option("url",mssql_cot_url) \
    .option("user","mehassan") \
    .option("password",'password') \
    .option("dbtable",cot_btc_table) \
    .load()

In [17]:
cot_data.show(n=3)

+-------------------+--------------------------+---------------------------+-----------------------+------------------------+-----------+--------------+
|               date|noncomm_positions_long_all|noncomm_positions_short_all|comm_positions_long_all|comm_positions_short_all|comm_spread|non_comm_spred|
+-------------------+--------------------------+---------------------------+-----------------------+------------------------+-----------+--------------+
|2021-04-06 00:00:00|                      5938|                       8230|                    504|                     204|        300|         -2292|
|2019-05-07 00:00:00|                      2946|                       3985|                      0|                      46|        -46|         -1039|
|2021-09-28 00:00:00|                      4539|                       5422|                    479|                     301|        178|          -883|
+-------------------+--------------------------+---------------------------+------

In [21]:
cot_data_preprocessed = cot_data.select("date","comm_spread")
cot_data_preprocessed = cot_data_preprocessed.withColumnRenamed("comm_spread","btc_com_spread")

In [28]:
cot_data_preprocessed.printSchema()

root
 |-- date: timestamp (nullable = true)
 |-- btc_com_spread: long (nullable = true)



In [34]:
crypto_data_refined.printSchema()
crypto_data_refined = crypto_data_refined.withColumn("date",col("date").cast("timestamp"))

root
 |-- date: string (nullable = true)
 |-- BTC: double (nullable = true)



In [35]:
crypto_data_refined.printSchema()

root
 |-- date: timestamp (nullable = true)
 |-- BTC: double (nullable = true)



In [12]:
crypto_data = spark.read.format("jdbc") \
    .option("url",mssql_url) \
    .option("user","mehassan") \
    .option("password",'password') \
    .option("dbtable",table) \
    .load()

In [18]:
crypto_data_refined = crypto_data.select("date","close")
crypto_data_refined = crypto_data_refined.withColumnRenamed("close","BTC")

In [19]:
crypto_data_refined.show(n=3)

+----------+---------+
|      date|      BTC|
+----------+---------+
|2021-08-02|  39152.3|
|2014-09-24|433.15178|
|2023-04-20| 28248.18|
+----------+---------+
only showing top 3 rows



In [36]:
processed_crypto_data = crypto_data_refined.join(cot_data_preprocessed,on ="date",how="left")

In [40]:
cot_data_preprocessed.count()

307

In [52]:
processed_crypto_data.select(sum("btc_com_spread").isNull()).collect() 

[Row((sum(btc_com_spread) IS NULL)=False)]

In [38]:
processed_crypto_data.count()

3762

In [5]:
news_data = spark.read.format("jdbc") \
    .option("url",workbench_url) \
    .option("user","mysql_user") \
    .option("password",'mu!tFL^fh5kE#6z8"4dyBs') \
    .option("dbtable","historical_with_sentiment") \
    .option("driver",driver) \
    .load()

In [16]:
stocks = news_data.select("symbol").distinct().rdd.flatMap(lambda x : x).collect()

In [44]:
stock_symbols = [
    'AAPL', 'MSFT', 'NVDA', 'META', 'AMZN', 'TSLA', 'GOOGL',
    'DBD', 'DSGX', 'GTLB', 'LOGI', 'CRSR',
    'LNG', 'SWN', 'APA', 'BTU', 'CL',
    'BMY', 'THC', 'TNDM',
    'MOS', 'AXTA', 'KOP',
    'SBLK', 'EME', 'DNOW',
]


In [47]:
set(stocks) - set(stock_symbols)

{'CVX', 'ON', 'XOM'}

In [30]:
individual_data = {}
for i in stocks:
    individual_data[i] = news_data.filter(news_data["symbol"] == i)
    individual_data[i] = individual_data[i].select("date","Close","sentiment")
    individual_data[i] = individual_data[i].withColumnRenamed("Close",i)

In [34]:
output_url = "jdbc:mysql://localhost:3306/big_data?permitMysqlSchema"

In [39]:
from pyspark.sql.utils import AnalysisException

In [50]:
stocks

['AAPL',
 'LOGI',
 'XOM',
 'BMY',
 'META',
 'LNG',
 'TSLA',
 'EME',
 'SWN',
 'AXTA',
 'CL',
 'SBLK',
 'CVX',
 'TNDM',
 'GTLB',
 'GOOGL',
 'APA',
 'ON',
 'MOS',
 'DBD',
 'DSGX',
 'KOP',
 'BTU',
 'THC',
 'AMZN',
 'MSFT',
 'CRSR',
 'DNOW',
 'NVDA']

In [51]:
individual_data = {}
try :
    for i in stock_symbols:
        individual_data[i] = news_data.filter(news_data["symbol"] == i)
        individual_data[i] = individual_data[i].select("date","Close","sentiment")
        individual_data[i] = individual_data[i].withColumnRenamed("Close",i)
        if individual_data[i] != "XOM" or "ON" or "CVX":
            individual_data[i].write.format("jdbc") \
                .option("url",output_url) \
                .option("driver",driver) \
                .option("dbtable",f"{i}") \
                .option("user","root") \
                .option("password","password") \
                .save()
            print(i)
except AnalysisException as e:
    print(e)

AAPL
MSFT
NVDA
META
AMZN
TSLA
GOOGL
DBD
DSGX
GTLB
LOGI
CRSR
LNG
SWN
APA
BTU
CL
BMY
THC
TNDM
MOS
AXTA
KOP
SBLK
EME
DNOW


In [33]:
individual_data["AAPL"].tail(3)

[Row(date=datetime.date(2024, 2, 28), AAPL=181.4199981689453, sentiment='1'),
 Row(date=datetime.date(2024, 2, 29), AAPL=180.75, sentiment='1'),
 Row(date=datetime.date(2024, 3, 1), AAPL=179.66000366210938, sentiment='1')]

In [27]:
individual_data["AAPL"].filter(individual_data["AAPL"]["sentiment"] != np.nan).show(3)

+------+----------+-----------------+---------+---------+
|symbol|      date|            Close|   Volume|sentiment|
+------+----------+-----------------+---------+---------+
|  AAPL|2024-02-27|182.6300048828125| 54318900|        1|
|  AAPL|2024-02-28|181.4199981689453| 48953900|        1|
|  AAPL|2024-02-29|           180.75|136682600|        1|
+------+----------+-----------------+---------+---------+
only showing top 3 rows



In [29]:
type(individual_data["AAPL"])

pyspark.sql.dataframe.DataFrame

In [ ]:
for i in individual_data.keys():
    individual_data[i].select

In [ ]:
pandas_df = news_data.toPandas()

In [ ]:
pandas_df['text'][0]

In [ ]:
def sentiment_analysis(data):
    data["Sentiment"] = ""
    sentiment = []
    for i in data['text']:
        tokenized_news = tokenizer(i,return_tensors="tf")
        logits = model.predict(tokenized_news).logits
        probabilities = tf.nn.softmax(logits)
        sentiment.append(probabilities)
    data["Sentiment"] = sentiment
    return data

In [ ]:
data_sentiment = sentiment_analysis(pandas_df)

In [ ]:
news_df = data_sentiment[["symbol","Sentiment"]].sort_values(by="symbol")

In [ ]:
news_df.iloc[1,1]

In [ ]:
news_symbol = news_df["symbol"].unique()
news_symbol

In [ ]:
data_news = {}
for i in news_symbol:
    news = []
    news_for_symbol = news_df[news_df["symbol"] == i]
    for sentiment in news_for_symbol["Sentiment"]:
        news.append(sentiment)
    data_news[i] = news 

In [ ]:
average_sentiment = {}
for symbol, tensors in data_news.items():
    mean_tensor = tf.reduce_mean(tensors,axis=0)
    average_sentiment[symbol] = np.argmax(tf.nn.softmax(mean_tensor))

In [ ]:
average_sentiment

In [ ]:
tf.nn.softmax(logits)

In [ ]:
predicted_class_id = int(tf.math.argmax(logits, axis=-1)[0])
model.config.id2label[predicted_class_id]
